In [1]:
import torch
import sys

from collections import OrderedDict
from torch import nn
from torchvision.models.inception import BasicConv2d, InceptionA, InceptionB, InceptionC

In [2]:
sys.path.append('../src')

In [3]:
from models.inception import Inception3_Encoder
from models.style_augment import TransformNet

In [4]:
# model = Inception3_Encoder()
# cp = torch.load('../../style-augmentation/styleaug/checkpoints/checkpoint_stylepredictor.pth')
# model.load_state_dict(cp['state_dict_stylepredictor'])

In [5]:
device = torch.device('cuda:1')

# x = torch.rand(6, 3, 299, 299).to(device)
# model = model.to(device)

In [6]:
net = TransformNet()

In [7]:
net = net.to(device)

In [8]:
N = 8
S = 512

x = torch.randn(N, 3, S, S).to(device)
y = torch.randn(N, 100).to(device)

In [9]:
net(x, y).shape

torch.Size([8, 3, 512, 512])

In [10]:
cp = torch.load('../../style-augmentation/styleaug/checkpoints/checkpoint_transformer.pth')

In [11]:
def port_transformer_weights(source):
    dest = dict()
    
    # encoder
    for i in range(3):
        for name in ['weight', 'bias']:
            name_source = 'layers.%d.conv.%s' % (i, name)
            name_dest = 'encoder.%d.conv.%s' % (i, name)
            dest[name_dest] = source[name_source]
            
    # bottleneck
    source_layers = ['conv', 'fc_beta', 'fc_gamma']
    dest_layers = ['layers.conv', 'beta', 'gamma']
    
    for i in range(5):
        for j in range(2):
            for source_layer, dest_layer in zip(source_layers, dest_layers):
                for name in ['weight', 'bias']:
                    name_source = 'layers.{}.{}.{}'.format(i + 3, source_layer + str(j +1 ), name)
                    name_dest = 'layers.{}.conv{}.{}.{}'.format(i, j + 1, dest_layer, name)
                    dest[name_dest] = source[name_source]
                    
    # decoder
    for i in range(3):
        for source_layer, dest_layer in zip(source_layers, dest_layers):
                for name in ['weight', 'bias']:
                    name_source = 'layers.{}.{}.{}'.format(i + 8, source_layer, name)
                    name_dest = 'layers.{}.{}.{}'.format(i + 5, dest_layer, name)
                    dest[name_dest] = source[name_source]
                    
    return dest

In [12]:
weights = port_transformer_weights(cp['state_dict_ghiasi'])

In [13]:
net.load_state_dict(weights)

<All keys matched successfully>